## Первое исследование

Сравнение Москвы и Петербурга окружено мифами. Например:

Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
Петербург — культурная столица, со своими вкусами.
На данных Яндекс.Музыки вы сравните поведение пользователей двух столиц.

#### Цель исследования   — проверьте три гипотезы:

Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

#### Ход исследования 

Данные о поведении пользователей вы получите из файла yandex_music_project.csv. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.

Вы проверите данные на ошибки и оцените их влияние на исследование. Затем, на этапе предобработки вы поищете возможность исправить самые критичные ошибки данных.

Таким образом, исследование пройдёт в три этапа:

- Обзор данных.
- Предобработка данных.
- Проверка гипотез.
- Обзор данных
- Составьте первое представление о данных Яндекс.Музыки.



### Обзор данных

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('data/yandex_music_project.csv')

In [3]:
data.head(5)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Согласно документации к данным:

- userID — идентификатор пользователя;
- Track — название трека;
- artist — имя исполнителя;
- genre — название жанра;
- City — город пользователя;
- time — время начала прослушивания;
- Day — день недели.

### Предобработка данных

In [5]:
data.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [6]:
data = data.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'}) 
data.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

In [7]:
data.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Данные в столбцах трек и исполнитель не особо важны для достижения цели выше. Поэтому пропуски в них можно заполнить как unknown. 

In [8]:
data[['track', 'artist', 'genre']] = data[['track', 'artist', 'genre']].fillna('unknown') 
data.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

Работа с дубликатами

In [9]:
print(data.duplicated().sum()) 

3826


In [10]:
data = data.drop_duplicates().reset_index(drop=True) 
print(data.duplicated().sum()) 

0


Неявные дубликаты

In [11]:
print(data['genre'].unique())   # hip, hop, hip-hop.
len(data['genre'].unique())

['rock' 'pop' 'folk' 'dance' 'rusrap' 'ruspop' 'world' 'electronic'
 'unknown' 'alternative' 'children' 'rnb' 'hip' 'jazz' 'postrock' 'latin'
 'classical' 'metal' 'reggae' 'tatar' 'blues' 'instrumental' 'rusrock'
 'dnb' 'türk' 'post' 'country' 'psychedelic' 'conjazz' 'indie'
 'posthardcore' 'local' 'avantgarde' 'punk' 'videogame' 'techno' 'house'
 'christmas' 'melodic' 'caucasian' 'reggaeton' 'soundtrack' 'singer' 'ska'
 'shanson' 'ambient' 'film' 'western' 'rap' 'beats' "hard'n'heavy"
 'progmetal' 'minimal' 'contemporary' 'new' 'soul' 'holiday' 'german'
 'tropical' 'fairytail' 'spiritual' 'urban' 'gospel' 'nujazz' 'folkmetal'
 'trance' 'miscellaneous' 'anime' 'hardcore' 'progressive' 'chanson'
 'numetal' 'vocal' 'estrada' 'russian' 'classicmetal' 'dubstep' 'club'
 'deep' 'southern' 'black' 'folkrock' 'fitness' 'french' 'disco'
 'religious' 'hiphop' 'drum' 'extrememetal' 'türkçe' 'experimental' 'easy'
 'metalcore' 'modern' 'argentinetango' 'old' 'breaks' 'eurofolk'
 'stonerrock' 'indus

290

In [12]:
data['genre'] = data['genre'].replace('hop', 'hip-hop')
data['genre'] = data['genre'].replace('hip', 'hip-hop')
data['genre'] = data['genre'].replace('электроника', 'electronic')

### Проверка гипотез

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверьте это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

- Разделите пользователей Москвы и Санкт-Петербурга
- Сравните, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.

In [13]:
spb_users = data[data['city'] == 'Saint-Petersburg'].reset_index(drop=True)   #18512 пользователей
msc_users = data[data['city'] == 'Moscow'].reset_index(drop=True)  # 42741 пользователей

In [14]:
spb_users_mon = spb_users[spb_users['day'] == 'Monday']['user_id'].count()  
spb_users_wen = spb_users[spb_users['day'] == 'Wednesday']['user_id'].count()   
spb_users_fri = spb_users[spb_users['day'] == 'Friday']['user_id'].count()   

msc_users_mon = msc_users[msc_users['day'] == 'Monday']['user_id'].count()   
msc_users_wen = msc_users[msc_users['day'] == 'Wednesday']['user_id'].count()
msc_users_fri = msc_users[msc_users['day'] == 'Friday']['user_id'].count()

d = {
    'city': ['Moscow', 'Saint-Petersburg'],
    'Monday': [msc_users_mon, spb_users_mon],
    'Wednesday': [msc_users_wen, spb_users_wen],
    'Friday': [msc_users_fri, spb_users_fri]
}

result = pd.DataFrame(data=d)
result

,city,Monday,Wednesday,Friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.  - ВЕРНО
Пик в Москве - понедельник и пятница
Пик в Питере - среда

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Сохраните таблицы с данными в две переменные:

по Москве — в moscow_general;
по Санкт-Петербургу — в spb_general.

In [15]:
def day_or_night(dataset):
    lenght = dataset.shape[0]
    
    for i in range(0, lenght):
        dataset.loc[i, 'time'] =  int(dataset.loc[i, 'time'].replace(':', '')[:4])
        
    day = dataset[(dataset['time'] >= 700) & (dataset['time'] <= 1100)]
    night = dataset[(dataset['time'] >= 1700) & (dataset['time'] <= 2300)]

    return day, night

In [16]:
spb_day, spb_night = day_or_night(spb_users)
msc_day, msc_night = day_or_night(msc_users)

In [18]:
# Топ 10 жанров в СПб утром понедельника

spb_day_mon = spb_day[spb_day['day'] == "Monday"]
spb_general_1 = spb_day_mon.groupby('genre')['genre'].count().sort_values(ascending=False).head(10)

spb_general_1

genre
pop            218
dance          182
rock           162
electronic     147
hip-hop         79
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [19]:
# Топ 10 жанров в СПб вечером пятницы

spb_night_fri = spb_night[spb_night['day'] == "Friday"]
spb_general_2 = spb_night_fri.groupby('genre')['genre'].count().sort_values(ascending=False).head(10)

spb_general_2

genre
pop            256
electronic     216
rock           216
dance          210
hip-hop         94
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

In [20]:
# Топ 10 жанров в МСК утром понедельника

msc_day_mon = msc_day[msc_day['day'] == "Monday"]
msc_general_1 = msc_day_mon.groupby('genre')['genre'].count().sort_values(ascending=False).head(10)

msc_general_1

genre
pop            781
dance          549
electronic     480
rock           474
hip-hop        282
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [21]:
# Топ 10 жанров в MCK вечером пятницы

msc_night_fri = msc_night[msc_night['day'] == "Friday"]
msc_general_2 = msc_night_fri.groupby('genre')['genre'].count().sort_values(ascending=False).head(10)

msc_general_2

genre
pop            713
rock           517
dance          495
electronic     482
hip-hop        267
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

В Москве пропущенных значений оказалось так много, что значение 'unknown' заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:

Пользователи слушают похожую музыку в начале недели и в конце.
Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.
Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные данные о жанрах.

Гипотеза 3: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве. А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [25]:
# Самые популярные жанры по Москве 

msc_users.groupby('genre')['genre'].count().sort_values(ascending=False).head(10)

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hip-hop        2043
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

In [26]:
# Самые популярные жанры по СПб

spb_users.groupby('genre')['genre'].count().sort_values(ascending=False).head(10)

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hip-hop         934
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

Гипотеза частично подтвердилась:

Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге.